In [ ]:
%cd A:/Helmet detection

In [ ]:
import cv2
import numpy as np
import os
import imutils
from tensorflow.keras.models import load_model
import time

os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

model = load_model("yolo_model.h5")
print('model loaded!!!')

cap = cv2.VideoCapture(0)  # Use webcam, change to the appropriate index if you have multiple cameras
COLORS = [(0, 0, 255)]  # Red color only

fourcc = cv2.VideoWriter_fourcc(*"XVID")  # open-source MPEG-4 video codec
writer = cv2.VideoWriter('output.avi', fourcc, 5, (888, 500))

# Folder to save cropped images
output_folder = 'output_images'
os.makedirs(output_folder, exist_ok=True)

def helmet_or_nohelmet(helmet_roi):
    try:
        helmet_roi = cv2.resize(helmet_roi, (224, 224))
        helmet_roi = np.array(helmet_roi, dtype='float32')
        helmet_roi = helmet_roi.reshape(1, 224, 224, 3)
        helmet_roi = helmet_roi / 255.0
        return int(model.predict(helmet_roi)[0][0])
    except:
        pass

layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

previous = 0

def control_traffic_signal(total_no_helmet_count, all_wearing_helmet, elapsed_time, red_signal_duration):
    global previous
    cycle_time = 36

    if all_wearing_helmet:
        red_signal_duration = 0
        green_time = 15
        yellow_time = 6

        remaining_time = elapsed_time % cycle_time

        if remaining_time < green_time:
            signal_color = (0, 255, 0)  # Green
            signal_text = f"Green Signal: {green_time - remaining_time} seconds"
            signal_image = cv2.imread("GREEN SIGNAL.jpg")
        elif remaining_time < green_time + yellow_time:
            signal_color = (0, 255, 255)  # Yellow
            signal_text = f"Yellow Signal: {green_time + yellow_time - remaining_time} seconds"
            signal_image = cv2.imread("YELLOW SIGNAL.jpg")
        else:
            signal_color = (0, 0, 255)  # Red
            signal_text = f"Red Signal: {cycle_time - remaining_time} seconds"
            signal_image = cv2.imread("RED SIGNAL.jpg")

        cv2.putText(signal_image, signal_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, signal_color, 2, cv2.LINE_AA)

        cv2.imshow('Traffic Signal', signal_image)
        cv2.waitKey(1)  # Allow the GUI to process events

        return elapsed_time + 1, max(0, red_signal_duration - 1)  # Increment elapsed time,
    else:
        signal_color = (0, 0, 255)  # Red
        signal_text = f" PLEASE WEAR HELMET Red Signal: {max(0,red_signal_duration)} seconds"
        signal_image = cv2.imread("RED SIGNAL.jpg")
        
        if total_no_helmet_count > 0:
            if red_signal_duration == 0:
                red_signal_duration = 5*total_no_helmet_count
                previous = 5 * total_no_helmet_count
            else:
                # Increment red signal duration by 5 seconds for every person without a helmet
                red_signal_duration = ((5 * total_no_helmet_count) - previous) + red_signal_duration
                red_signal_duration = max(0, red_signal_duration)
                previous = 5 * total_no_helmet_count

        if elapsed_time >= 20:
            green_signal_duration = 10
            remaining_green_time = (elapsed_time - 20) % (green_signal_duration + cycle_time)

            if remaining_green_time < green_signal_duration:
                signal_color = (0, 255, 0)  # Green
                signal_text = f"Passing Green Signal: {green_signal_duration - remaining_green_time} seconds"
                signal_image = cv2.imread("GREEN SIGNAL.jpg")
                red_signal_duration = 0  # Reset red signal duration when green signal is released
                previous = 0
            else:
                signal_text = f"PLEASE WEAR HELMET Red Signal: {red_signal_duration} seconds"
                print("2 minutes have passed. Resuming normal operation")
        else:
            signal_text = f"PLEASE WEAR HELMET Red Signal: {red_signal_duration} seconds"
            print("Not all individuals have helmets. Traffic signal: Red")

        cv2.putText(signal_image, signal_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, signal_color, 2, cv2.LINE_AA)
        cv2.imshow('Traffic Signal', signal_image)
        cv2.waitKey(1)  # Allow the GUI to process events

        return elapsed_time + 1, max(0, red_signal_duration - 1)  # Increment elapsed time, update red signal duration


red_signal_duration = 0
elapsed_time = 0
while True:
    # Clear the list for each frame
    no_helmet_images = []

    ret, img = cap.read()
    if not ret:
        break

    img = imutils.resize(img, height=500)
    height, width = img.shape[:2]

    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    net.setInput(blob)
    outs = net.forward(output_layers)

    confidences = []
    boxes = []
    classIds = []

    total_helmet_count = 0
    total_no_helmet_count = 0
    all_wearing_helmet = True

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)

                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                classIds.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            helmet_roi = img[max(0, y):max(0, y) + max(0, h) // 4, max(0, x):max(0, x) + max(0, w)]

            if classIds[i] == 0:
                if helmet_or_nohelmet(helmet_roi) == 1:
                    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    total_helmet_count += 1
                else:
                    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 2)
                    all_wearing_helmet = False

                    if 0 <= y < img.shape[0] and 0 <= x < img.shape[1]:
                        image_path = os.path.join(output_folder, f'no_helmet_{total_no_helmet_count}.jpg')
                        cropped_image = img[y:y+h, x:x+w]
                        no_helmet_images.append(cropped_image)
                        total_no_helmet_count += 1

    elapsed_time, red_signal_duration = control_traffic_signal(total_no_helmet_count, all_wearing_helmet, elapsed_time, red_signal_duration)

     # Display the original frame with the count of people without helmets
    cv2.putText(img, f'Total no-helmet count: {total_no_helmet_count}', (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.imshow('Original Frame with Count', img)
    cv2.waitKey(1)  # Allow the GUI to process events

   
    # Display cropped images of riders without helmets in a single pop-up window
    if no_helmet_images:
        print("Displaying all cropped images without helmets in one pop-up:")
        max_height = max(image.shape[0] for image in no_helmet_images)
        resized_images = [cv2.resize(image, (int(image.shape[1] * max_height / image.shape[0]), max_height)) for image in no_helmet_images]
        collage = np.concatenate(resized_images, axis=1)  # Concatenate resized images horizontally
        cv2.imshow('Cropped Images Collage', collage)
    else:
        # Display "OFF SCREEN.jpeg" when there are no individuals without helmets
        off_screen_image = cv2.imread("OFF SCREEN.jpeg")
        cv2.imshow('Cropped Images Collage', off_screen_image)



    time.sleep(2)  # Wait for 1 second

# Release resources
writer.release()
cap.release()
cv2.destroyAllWindows()